### JSON Schemas

https://json-schema.org/

We've seen that one of the issues we have when trying to deserialize JSON is having a known schema, so that we can do more than just the standards.

In [2]:
person_schema = {
    "type": "object",
    "properties": {
        "firstName": {"type": "string"},
        "middleInitial": {"type": "string"} ,
        "lastName": {"type": "string"},
        "age": {"type": "number"}
    }
}

In [3]:
p1 = '''
{
    "firstName": "John",
    "middleInitial": "M",
    "lastName": "Cleese",
    "age": 79
}
'''

In [4]:
p2 = '''
{
    "firstName": "John",
    "middleInitial": 100,
    "lastName": "Cleese",
    "age": "Unknown"
}
'''

In [5]:
p3 = '''
{
    "firstName": "John",
    "age": -10.5
}
'''

In [14]:
person_schema = {
    "type": "object",
    "properties": {
        "firstName": {
            "type": "string",
            "minLength": 1
        },
        "middleInitial": {
            "type": "string",
            "minLength": 1,
            "maxLength": 1
        } ,
        "lastName": {
            "type": "string",
            "minLength": 1        
        },
        "age": {
            "type": "integer",
            "minimum": 0
        },
        "eyeColor": {
            "type": "string",
            "enum": ["amber", "blue", "brown", "gray",
                     "green", "hazel", "red", "violet"]
        }
    },
    "required": ["firstName", "lastName"]
}

In [15]:
pip install jsonschema

In [16]:
from jsonschema import validate
from jsonschema.exceptions import ValidationError
from json import loads, dumps, JSONDecodeError

In [17]:
json_doc = p1

print(json_doc)

try:
    validate(loads(json_doc), person_schema)
except JSONDecodeError as ex:
    print(f'Invalid JSON: {ex}')
except ValidationError as ex:
    print(f'Validation error: {ex}')
else:
    print('JSON is valid and conforms to schema')


{
    "firstName": "John",
    "middleInitial": "M",
    "lastName": "Cleese",
    "age": 79
}

JSON is valid and conforms to schema


In [18]:
json_doc = p2

print(json_doc)

try:
    validate(loads(json_doc), person_schema)
except JSONDecodeError as ex:
    print(f'Invalid JSON: {ex}')
except ValidationError as ex:
    print(f'Validation error: {ex}')
else:
    print('JSON is valid and conforms to schema')


{
    "firstName": "John",
    "middleInitial": 100,
    "lastName": "Cleese",
    "age": "Unknown"
}

Validation error: 100 is not of type 'string'

Failed validating 'type' in schema['properties']['middleInitial']:
    {'maxLength': 1, 'minLength': 1, 'type': 'string'}

On instance['middleInitial']:
    100


In [19]:
json_doc = p3

print(json_doc)

try:
    validate(loads(json_doc), person_schema)
except JSONDecodeError as ex:
    print(f'Invalid JSON: {ex}')
except ValidationError as ex:
    print(f'Validation error: {ex}')
else:
    print('JSON is valid and conforms to schema')


{
    "firstName": "John",
    "age": -10.5
}

Validation error: 'lastName' is a required property

Failed validating 'required' in schema:
    {'properties': {'age': {'minimum': 0, 'type': 'integer'},
                    'eyeColor': {'enum': ['amber',
                                          'blue',
                                          'brown',
                                          'gray',
                                          'green',
                                          'hazel',
                                          'red',
                                          'violet'],
                                 'type': 'string'},
                    'firstName': {'minLength': 1, 'type': 'string'},
                    'lastName': {'minLength': 1, 'type': 'string'},
                    'middleInitial': {'maxLength': 1,
                                      'minLength': 1,
                                      'type': 'string'}},
     'required': ['firstName', 'last

In [21]:
from jsonschema import Draft4Validator

validator = Draft4Validator(person_schema)

In [24]:
json_doc = p1

for error in validator.iter_errors(loads(json_doc)):
    print(error, end='\n-----------------------\n')

In [25]:
json_doc = p2

for error in validator.iter_errors(loads(json_doc)):
    print(error, end='\n-----------------------\n')

100 is not of type 'string'

Failed validating 'type' in schema['properties']['middleInitial']:
    {'maxLength': 1, 'minLength': 1, 'type': 'string'}

On instance['middleInitial']:
    100
-----------------------
'Unknown' is not of type 'integer'

Failed validating 'type' in schema['properties']['age']:
    {'minimum': 0, 'type': 'integer'}

On instance['age']:
    'Unknown'
-----------------------


In [26]:
json_doc = p3

for error in validator.iter_errors(loads(json_doc)):
    print(error, end='\n-----------------------\n')

-10.5 is not of type 'integer'

Failed validating 'type' in schema['properties']['age']:
    {'minimum': 0, 'type': 'integer'}

On instance['age']:
    -10.5
-----------------------
-10.5 is less than the minimum of 0

Failed validating 'minimum' in schema['properties']['age']:
    {'minimum': 0, 'type': 'integer'}

On instance['age']:
    -10.5
-----------------------
'lastName' is a required property

Failed validating 'required' in schema:
    {'properties': {'age': {'minimum': 0, 'type': 'integer'},
                    'eyeColor': {'enum': ['amber',
                                          'blue',
                                          'brown',
                                          'gray',
                                          'green',
                                          'hazel',
                                          'red',
                                          'violet'],
                                 'type': 'string'},
                    'firstName': 

In [27]:
p4 = '''
{
   "firstName": "john",
   "middleInitial": null,
   "lastName": "Cleese",
   "eyeColor": "blue-gray"
}
'''

In [28]:
json_doc = p4

for error in validator.iter_errors(loads(json_doc)):
    print(error, end='\n-----------------------\n')

None is not of type 'string'

Failed validating 'type' in schema['properties']['middleInitial']:
    {'maxLength': 1, 'minLength': 1, 'type': 'string'}

On instance['middleInitial']:
    None
-----------------------
'blue-gray' is not one of ['amber', 'blue', 'brown', 'gray', 'green', 'hazel', 'red', 'violet']

Failed validating 'enum' in schema['properties']['eyeColor']:
    {'enum': ['amber',
              'blue',
              'brown',
              'gray',
              'green',
              'hazel',
              'red',
              'violet'],
     'type': 'string'}

On instance['eyeColor']:
    'blue-gray'
-----------------------


In [29]:
p5 = '''
{
   "firstName": "john",
   "lastName": "Cleese",
   "eyeColor": "blue"
}
'''

In [30]:
json_doc = p5

for error in validator.iter_errors(loads(json_doc)):
    print(error, end='\n-----------------------\n')